In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import logging
logging.basicConfig(level=logging.INFO)

from myTorch.task.copying_memory import CopyingMemoryData
from myTorch.memnets.load import load_experiment
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
model = "/mnt/data/chinna/output/copying_memory_task/flatmemory_m64_k4_h80_linear/current"

In [ ]:
def example_memory(save_dir, time_lag, seed):
    experiment, model, data_iterator, device, config = load_experiment(save_dir)
    config.time_lag = time_lag
    config.batch_size = 1
    config.seed = seed
    new_data_iterator = CopyingMemoryData(seq_len=config.seq_len, time_lag=config.time_lag,
                                          batch_size=config.batch_size, seed=config.seed)
    data_iterator = new_data_iterator
    
    data = data_iterator.next()
    seqloss = 0

    model.reset_hidden(batch_size=1)

    hiddens = []

    for i in range(0, data["datalen"]):

        x = torch.from_numpy(np.asarray(data['x'][i])).to(device)
        y = torch.from_numpy(np.asarray(data['y'][i])).to(device)
        mask = float(data["mask"][i])

        model.optimizer.zero_grad()

        output = model(x)
        hiddens.append(model._h_prev[0]["memory"])
        if config.task == "copying_memory":
            loss = F.torch.nn.functional.cross_entropy(output, y.squeeze(1))
        else:
            loss = F.binary_cross_entropy_with_logits(output, y)

        seqloss += (loss * mask)

    seqloss /= sum(data["mask"])
    print(seqloss)

    return hiddens
    

In [ ]:
memory = example_memory(model, time_lag = 100, seed=11)

x = len(memory)
y = len(memory[0][0].detach().numpy())

matrix = np.zeros((x,y))
for i in range(0,x):
    matrix[i] = memory[i][0].detach().numpy()
plt.subplots(figsize=(20,15))
ax = sns.heatmap(matrix)


In [ ]:
new_matrix = np.zeros((x,y))

for i in range(0,x):
    if i==0:
        new_matrix[i] = matrix[i]
    else:
        new_matrix[i] = matrix[i] - matrix[i-1]
plt.subplots(figsize=(20,15))
ax = sns.heatmap(new_matrix)